In [1]:
import pandas as pd

import plotly as pl
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout
from plotly.graph_objs import *
import numpy as np
import cufflinks as cf

import folium
from folium.plugins import MarkerCluster
from folium import plugins
from folium.plugins import HeatMap
from IPython.display import HTML
import folium
from IPython.html import widgets
from IPython.display import display, Image, HTML, clear_output

import html5lib
from IPython.display import HTML
import requests
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString, Tag

from scipy import stats

from datetime import date
import calendar


print ("plotly ",__version__)
init_notebook_mode(connected=True)


plotly  1.12.12


/Users/srs/anaconda/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning:

The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.



In [2]:
HTML('<iframe src=http://web.mta.info/developers/turnstile.html width=700 height=500></iframe>')

In [3]:
url_to_scrape = ('http://web.mta.info/developers/turnstile.html')
r = requests.get(url_to_scrape)
soup = BeautifulSoup(r.text)

sels = '#contentbox > div > div > a'
data = soup.select(sels)
b = []
c = []
for i in range(len(data)):
    b.append(str(data[i]))
    c.append(b[i].partition('"')[-1].rpartition('"')[0])
url_temp = 'http://web.mta.info/developers/{clist}'
url = []
for name in c:
    url.append( url_temp.format(clist=name))
url[:4]

/Users/srs/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/srs/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")




['http://web.mta.info/developers/data/nyct/turnstile/turnstile_170107.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161231.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161224.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161217.txt']

In [4]:
main_df = pd.DataFrame()
for numbers in url[:4]:

    df= pd.read_csv(numbers,
            parse_dates=[['TIME','DATE']], 
            infer_datetime_format= True,
            keep_date_col = True) 
    df.columns = [n.lower().strip() for n in df.columns]
    df['total_entries']= df.groupby(['c/a','unit','scp','station'])['entries'].diff()
    df['total_exits']= df.groupby(['c/a','unit','scp','station',])['exits'].diff()
    df.total_entries.fillna(0, inplace = True)
    df = df[np.abs(df.total_entries-df.total_entries.mean())<=(3*df.total_entries.std())]
    df = df [df.total_entries>=0]
    df = df [df.total_exits>=0]
    
    if main_df.empty:
        main_df = df
    else:
        
        main_df = main_df.append(df)

main_df['hour'] = main_df.time_date.dt.hour
main_df['day'] = main_df.time_date.dt.dayofweek
main_df['day'].replace([0,1,2,3,4,5,6],['Monday',
                                        'Tuesday',
                                        'Wednesday',
                                        'Thursday',
                                        'Friday',
                                        'Saturday',
                                        'Sunday'],inplace=True)
main_df.head()

,time_date,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,total_entries,total_exits,hour,day
1,2016-12-31 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389,19.0,11.0,7,Saturday
2,2016-12-31 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441,79.0,52.0,11,Saturday
3,2016-12-31 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502,327.0,61.0,15,Saturday
4,2016-12-31 19:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543,447.0,41.0,19,Saturday
5,2016-12-31 23:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,23:00:00,REGULAR,5992638,2028572,220.0,29.0,23,Saturday


In [6]:
df2 = main_df.groupby(['station'],)[['total_entries','total_exits']].sum()
df3 = df2.sort(['total_entries'],ascending=False)
df4 = df2.sort(['total_exits'],ascending=False)

/Users/srs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)

/Users/srs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [7]:
iplot((df3.sort(['total_entries'],ascending=False)).iplot(asFigure=True,
               kind = 'scatter',
               xTitle = 'Station name',
               yTitle = 'Total number of passenger',
               title = 'Total number of passenger'))

/Users/srs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [ ]:
iplot(df3.iplot(asFigure=True,
                 kind='bar',
                 xTitle='Days Of TheWeek',
                 yTitle='Total number of travel',
                 title=''))

In [43]:
mask = ((main_df["c/a"] == "A002") & 
        (main_df["unit"] == "R051") & 
        (main_df["scp"] == "02-00-00") & 
        (main_df["station"] == "59 ST"))
main_df[mask].head()

,time_date,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,total_entries,total_exits,hour,day
1,2016-12-31 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389,19.0,11.0,7,Saturday
2,2016-12-31 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441,79.0,52.0,11,Saturday
3,2016-12-31 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502,327.0,61.0,15,Saturday
4,2016-12-31 19:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543,447.0,41.0,19,Saturday
5,2016-12-31 23:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,23:00:00,REGULAR,5992638,2028572,220.0,29.0,23,Saturday


In [ ]:
def test(x):
    for _ in range (7):
        if x == 0:
            return ('Monday')
        if x == 1:
            return ('Tuesday')
        if x == 2:
            return ('Wednesday')
        if x == 3:
            return ('Thursday')
        if x == 4:
            return ('Friday')
        if x == 5:
            return ('Saturday')
        if x == 6:
            return ('Sunday')
main_df['DaysOfTheWeek'] = main_df.day.apply(test)

In [39]:
main_df['day'].replace([0,1,2,3,4,5,6],['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],inplace=True)

In [40]:
main_df.head()

,time_date,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,total_entries,total_exits,hour,day
1,2016-12-31 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389,19.0,11.0,7,Saturday
2,2016-12-31 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441,79.0,52.0,11,Saturday
3,2016-12-31 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502,327.0,61.0,15,Saturday
4,2016-12-31 19:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543,447.0,41.0,19,Saturday
5,2016-12-31 23:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,23:00:00,REGULAR,5992638,2028572,220.0,29.0,23,Saturday


In [27]:
len(main_df)

749977

In [28]:
main_df.describe()

,entries,exits,total_entries,total_exits
count,7.499770e+05,7.499770e+05,749977.000000,749977.000000
mean,2.506762e+07,1.851440e+07,172.012971,129.802971
std,1.491262e+08,1.318430e+08,869.436303,271.637706
min,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,5.567570e+05,2.799070e+05,11.000000,9.000000
50%,2.636275e+06,1.521332e+06,73.000000,49.000000
75%,6.556358e+06,4.656541e+06,231.000000,153.000000
max,2.114952e+09,2.087387e+09,710664.000000,118058.000000


In [10]:
dates = '170107',
main_df = pd.DataFrame()
for numbers in dates:

    df= pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+numbers+'.txt',
            parse_dates=[['TIME','DATE']], 
            infer_datetime_format= True,
            keep_date_col = True) 
    
    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.append(df)

In [ ]:
datetime.datetime.strptime('January 11, 2010', '%B %d, %Y').strftime('%A')


In [71]:
main_df['hour'] = main_df.TIME_DATE.dt.hour
main_df['day'] = main_df.TIME_DATE.dt.dayofweek

In [86]:
def test(x):
    for _ in range (7):
        if x == 0:
            return ('Monday')
        if x == 1:
            return ('Tuesday')
        if x == 2:
            return ('Wednesday')
        if x == 3:
            return ('Thursday')
        if x == 4:
            return ('Friday')
        if x == 5:
            return ('Saturday')
        if x == 6:
            return ('Sunday')
main_df['DaysOfTheWeek'] = main_df.day.apply(test)

In [91]:
main_df['DaysOfTheWeek'] = main_df.day.apply(test)

In [80]:
def domain(x):
    for a in range (len (x)):
        if x[a] == '@':
            return(x[a+1:])
fac['domain']= fac.email.apply(domain)

['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [87]:
main_df.head()

,TIME_DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,toal_number,dates,hour,day,test
0,2016-12-31 03:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,03:00:00,REGULAR,5991546,2028378,NaN,12-31,3,5,Saturday
1,2016-12-31 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389,19.0,12-31,7,5,Saturday
2,2016-12-31 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441,79.0,12-31,11,5,Saturday
3,2016-12-31 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502,327.0,12-31,15,5,Saturday
4,2016-12-31 19:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543,447.0,12-31,19,5,Saturday


In [44]:
my_date = date.today()
calendar.day_name[my_date.weekday()]

'Tuesday'

In [93]:
df2 = main_df.groupby(['hour'],)[['toal_number']].sum()
df3 = main_df.groupby(['DaysOfTheWeek'],)[['toal_number']].sum()

In [89]:
df3

,toal_number
day,
0,2581732.0
1,71273935.0
2,102415563.0
3,5426254.0
4,17299835.0
5,2400027.0
6,2138569.0


In [94]:
iplot(df3.iplot(asFigure=True,
                 kind='bar',
                 xTitle='Days Of TheWeek',
                 yTitle='Total number of travel',
                 title=''))

In [79]:
iplot(df2.iplot(asFigure=True,
                 kind='bar',
                 xTitle='STATION-hour',
                 yTitle='Total number of travel',
                 title=''))

In [100]:
df2= main_df.groupby(['STATION']).sum()
df2.head()

,ENTRIES,EXITS,toal_number,hour,day
STATION,,,,,
1 AV,59424116250,61655427960,119342.0,5460,1260
103 ST,6972006631,27801218648,164877.0,6432,1944
103 ST-CORONA,2829037491,2661422082,113706.0,4907,1134
104 ST,204174320606,210168297198,17911.0,4881,1129
110 ST,1225823192,1210781063,68293.0,3254,748


In [131]:
from scipy import stats

main_df.toal_number.fillna(0, inplace = True)
df4 = main_df[np.abs(main_df.toal_number-main_df.toal_number.mean())<=(3*main_df.toal_number.std())]
df4 = df4[df4.toal_number>=0]

In [113]:
main_df.head()

,TIME_DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,toal_number,dates,hour,day,test,DaysOfTheWeek
0,2016-12-31 03:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,03:00:00,REGULAR,5991546,2028378,0.0,12-31,3,5,Saturday,Saturday
1,2016-12-31 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389,19.0,12-31,7,5,Saturday,Saturday
2,2016-12-31 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441,79.0,12-31,11,5,Saturday,Saturday
3,2016-12-31 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502,327.0,12-31,15,5,Saturday,Saturday
4,2016-12-31 19:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543,447.0,12-31,19,5,Saturday,Saturday


In [114]:
df4.head()

,TIME_DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,toal_number,dates,hour,day,test,DaysOfTheWeek
0,2016-12-31 03:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,03:00:00,REGULAR,5991546,2028378,0.0,12-31,3,5,Saturday,Saturday
1,2016-12-31 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389,19.0,12-31,7,5,Saturday,Saturday
2,2016-12-31 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441,79.0,12-31,11,5,Saturday,Saturday
3,2016-12-31 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502,327.0,12-31,15,5,Saturday,Saturday
4,2016-12-31 19:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543,447.0,12-31,19,5,Saturday,Saturday


In [132]:
df5= df4.groupby(['STATION']).sum()
df5.head()

,ENTRIES,EXITS,toal_number,hour,day
STATION,,,,,
1 AV,59424116250,61655427960,119342.0,5460,1260
103 ST,6972006631,27801218648,164877.0,6432,1944
103 ST-CORONA,2829037491,2661422082,113706.0,4907,1134
104 ST,82822261876,159043373592,20871.0,3898,930
110 ST,1225823192,1210781063,68293.0,3254,748


In [133]:
iplot(df5.iplot(asFigure=True,
                 kind='bar',
                 xTitle='STATION-hour',
                 yTitle='Total number of travel',
                 title=''))

In [161]:
df6= df4.groupby(['TIME_DATE']).sum()
df6.head()

,ENTRIES,EXITS,toal_number,hour,day
TIME_DATE,,,,,
2016-12-31 00:00:00,53976080441,41740453035,0.0,0,8660
2016-12-31 00:01:28,13723,191,0.0,0,5
2016-12-31 00:01:47,174376,58717,0.0,0,5
2016-12-31 00:04:04,102277,1344349,0.0,0,5
2016-12-31 00:04:15,106826,226811,0.0,0,5


In [165]:
iplot(df6.iplot(asFigure=True,
                 kind='line',
                 xTitle='Time and dates',
                 yTitle='Total number of travel',
                 title=''))

In [102]:
df2= main_df.groupby(['STATION']== 'CANAL ST').sum()
df2.head()

KeyError: False

In [101]:
iplot(df2.iplot(asFigure=True,
                 kind='bar',
                 xTitle='STATION-hour',
                 yTitle='Total number of travel',
                 title=''))

In [24]:
main_df['dates'] = main_df['TIME_DATE'].map(lambda x: x.strftime("%m-%d"))
main_df.tail()

,TIME_DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,toal_number,dates
194769,2017-01-06 04:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,04:00:00,REGULAR,5554,283,0.0,01-06
194770,2017-01-06 08:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,08:00:00,REGULAR,5554,283,0.0,01-06
194771,2017-01-06 12:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,12:00:00,REGULAR,5554,283,0.0,01-06
194772,2017-01-06 16:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,16:00:00,REGULAR,5554,283,0.0,01-06
194773,2017-01-06 20:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,20:00:00,REGULAR,5554,283,0.0,01-06


In [213]:
main_df['toal_number']= main_df.groupby(['DATE','C/A','UNIT','SCP','STATION'])['ENTRIES'].diff()
#main_df['EXITS_1']= main_df.groupby(['C/A','UNIT','SCP','STATION','TIME_DATE'])['EXITS'].diff()
main_df.head(10)

ValueError: 

In [220]:
df.columns

Index(['time_date', 'c/a', 'unit', 'scp', 'station', 'linename', 'division',
       'date', 'time', 'desc', 'entries', 'exits', 'toal_number'],
      dtype='object')

In [218]:
df.columns = [n.lower().strip() for n in df.columns]

In [221]:
df.columns = [n.lower().strip() for n in df.columns]
df['toal_number']= df.groupby(['c/a','unit','scp','station'])['entries'].diff()
df['exits_1']= df.groupby(['c/a','unit','scp','station'])['exits'].diff()
df.head(10)

,time_date,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,toal_number,exits_1
0,2016-12-17 03:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,03:00:00,REGULAR,5967477,2022101,NaN,NaN
1,2016-12-17 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,07:00:00,REGULAR,5967485,2022116,8.0,15.0
2,2016-12-17 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,11:00:00,REGULAR,5967553,2022233,68.0,117.0
3,2016-12-17 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,15:00:00,REGULAR,5967790,2022331,237.0,98.0
4,2016-12-17 19:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,19:00:00,REGULAR,5968186,2022421,396.0,90.0
5,2016-12-17 23:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,23:00:00,REGULAR,5968429,2022470,243.0,49.0
6,2016-12-18 03:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2016,03:00:00,REGULAR,5968519,2022492,90.0,22.0
7,2016-12-18 07:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2016,07:00:00,REGULAR,5968527,2022504,8.0,12.0
8,2016-12-18 11:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2016,11:00:00,REGULAR,5968571,2022583,44.0,79.0
9,2016-12-18 15:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2016,15:00:00,REGULAR,5968782,2022656,211.0,73.0


In [ ]:
df.groupby(['C/A','UNIT','SCP','STATION']).sum()

In [ ]:
main_df['dates'] = main_df['TIME_DATE'].map(lambda x: x.strftime("%m-%d"))
main_df.tail()

In [ ]:
df['toal_number']= df.groupby(['C/A','UNIT','SCP','STATION','dates'])['ENTRIES'].diff()
df.head(10)

http://stackoverflow.com/questions/28129674/python-pandas-converting-from-cumulative-values-to-deltas


In [ ]:
df2 = df.groupby(['C/A','UNIT','SCP','STATION','dates'], as_index = False)[['toal_number']].sum()

In [ ]:
df2.head()

In [ ]:
df3 =df.groupby(['STATION'])[['toal_number']].sum()

In [ ]:
df3.head(10)

In [ ]:
iplot(df3.iplot(asFigure=True,
                 kind='bar',
                 xTitle='STATION',
                 yTitle='toal_number',
                 title=''))


In [ ]:
#CategoryC = train.Category.value_counts()

iplot(df2.iplot(asFigure=True,
                 kind='bar',
                 xTitle='STATION',
                 yTitle='toal_number',
                 title=''))





In [ ]:
website = pd.read_html('http://web.mta.info/developers/turnstile.html')

In [11]:
HTML('<iframe src=http://web.mta.info/developers/turnstile.html width=700 height=500></iframe>')

In [134]:
url_to_scrape = ('http://web.mta.info/developers/turnstile.html')
r = requests.get(url_to_scrape)
soup = BeautifulSoup(r.text)


/Users/srs/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/srs/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")




In [135]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html lang="en">
<head>
<title>mta.info | Turnstile Data</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!--<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7">-->
<link href="/siteimages/favicon.ico" rel="shortcut icon"/>
<link href="/css/base.css" rel="stylesheet" type="text/css"/>
<link href="/css/grid.css" rel="stylesheet" type="text/css"/>
<link href="/css/topbar.css" rel="stylesheet" type="text/css"/>
<link href="/css/formalize.css" rel="stylesheet" type="text/css"/>
<!-- <link rel="stylesheet" type="text/css" href="/css/jquery.datepick.css"> -->
<!-- jQuery include should be at the top -->
<script language="javascript" src="/js/jquery-1.4.4.min.js" type="text/javascript"></script>
<link href="/css/template.css" rel="stylesheet" type="text/css"/>
<meta content="News" name="WT.cg_n"/>
<style media="all" type="text/css">
@import url(/mta/mtahq_custom_clean

In [6]:
url_to_scrape = ('http://web.mta.info/developers/turnstile.html')
r = requests.get(url_to_scrape)
soup = BeautifulSoup(r.text)
sels = '#contentbox > div > div > a'
data = soup.select(sels)
data

/Users/srs/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/srs/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")




In [9]:
sels = '#contentbox > div > div > a'
data = soup.select(sels)
b = []
c = []
for i in range(len(data)):
    b.append(str(data[i]))
    c.append(b[i].partition('"')[-1].rpartition('"')[0])
url_temp = 'http://web.mta.info/developers/{clist}'
url = []
for name in c:
    url.append( url_temp.format(clist=name))
url[:3]

['http://web.mta.info/developers/data/nyct/turnstile/turnstile_170107.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161231.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161224.txt']

In [151]:
b = []
c = []
for i in range(len(data)):
    b.append(str(data[i]))
    c.append(b[i].partition('"')[-1].rpartition('"')[0])
url_temp = 'http://web.mta.info/developers/{clist}'
url = []
for name in c:
    url.append( url_temp.format(clist=name))

In [157]:
url_temp = 'http://web.mta.info/developers/{clist}'
url = []
for name in c:
    url.append( url_temp.format(clist=name))
    

In [166]:
url[:3]

['http://web.mta.info/developers/data/nyct/turnstile/turnstile_170107.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161231.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161224.txt']

In [ ]:
data = soup.select("br")
data

In [136]:
data = soup.find_all("div", class_='span-84 last')
data

[<div class="span-84 last">
 <h2>Data Files</h2>
 <br/>
 <a href="data/nyct/turnstile/turnstile_170107.txt">Saturday, January 07, 2017</a><br/><a href="data/nyct/turnstile/turnstile_161231.txt">Saturday, December 31, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161224.txt">Saturday, December 24, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161217.txt">Saturday, December 17, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161210.txt">Saturday, December 10, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161203.txt">Saturday, December 03, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161126.txt">Saturday, November 26, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161119.txt">Saturday, November 19, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161112.txt">Saturday, November 12, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161105.txt">Saturday, November 05, 2016</a><br/><a href="data/nyct/turnstile/turnstile_161029.txt">Saturday, October 29, 2

In [ ]:
data

In [ ]:
for br in data:
    next = br.nextSibling
    if not (next and isinstance(next,NavigableString)):
        continue
    next2 = next.nextSibling
    if next2 and isinstance(next2,Tag) and next2.name == 'br':
        text = str(next).strip()
        if text:
            print ("Found:", next)

In [ ]:
for th in data:
    th.find_all(text='a')

In [ ]:
th_all = data.find_all('br')
result = []
for br in th_all:
    result.extend(th.find_all(text='A'))

In [ ]:
for br in data.find_all('br'):
    next = br.nextSibling
    if not (next and isinstance(next,NavigableString)):
        continue
    next2 = next.nextSibling
    if next2 and isinstance(next2,Tag) and next2.name == 'br':
        text = str(next).strip()
        if text:
            print ("Found:", next)

In [ ]:
next2

In [ ]:
lobbying = {}
for element in data:
    lobbying[element.get_text()] = {}
lobbying

In [ ]:
for element in data:
    lobbying[element.get_text()]["link"] =  element.a["href"]

In [ ]:
lobbying

In [ ]:
type(soup)

In [ ]:
input = '''<br />
Important Text 1
<br />
<br />
Not Important Text
<br />
Important Text 2
<br />
Important Text 3
<br />
<br />
Non Important Text
<br />
Important Text 4
<br />'''

soup = BeautifulSoup(input)

for br in soup.find_all('br'):
    next = br.nextSibling
    if not (next and isinstance(next,NavigableString)):
        continue
    next2 = next.nextSibling
    if next2 and isinstance(next2,Tag) and next2.name == 'br':
        text = str(next).strip()
        if text:
            print ("Found:", next)

In [ ]:
soup

In [ ]:
next

In [ ]:
soup.findAll('br')

In [ ]:
input = '''<br />
Important Text 1
<br />
<br />
Not Important Text
<br />
Important Text 2
<br />
Important Text 3
<br />
<br />
Non Important Text
<br />
Important Text 4
<br />'''

soup = BeautifulSoup(input)

for br in soup.findAll('br'):
    next = br.nextSibling
    if not (next and isinstance(next,NavigableString)):
        continue
    next2 = next.nextSibling
    if next2 and isinstance(next2,Tag) and next2.name == 'br':
        text = str(next).strip()
        if text:
            print ("Found:", next)

In [ ]:

 a
inmates_links = []
 
for table_row in soup.select(".inmatesList tr"):
	table_cells = table_row.findAll('td')

	if len(table_cells) > 0:
		relative_link_to_inmate_details = table_cells[0].find('a')['href']
		absolute_link_to_inmate_details = url_to_scrape + relative_link_to_inmate_details
		inmates_links.append(absolute_link_to_inmate_details)